In [1]:
 from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.


Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'XMGFAGKBMEINJ4SMM4RTPAPGWBBDBE2S4OOGF1PUFRZJYBU1' # your Foursquare ID
CLIENT_SECRET = '5PSGNJQPGPQQFQPZB02CJAPVRFHPLRJF52PQTE0Q23FEJANR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XMGFAGKBMEINJ4SMM4RTPAPGWBBDBE2S4OOGF1PUFRZJYBU1
CLIENT_SECRET:5PSGNJQPGPQQFQPZB02CJAPVRFHPLRJF52PQTE0Q23FEJANR


In [3]:
address = 'saint john' # location in which i am intrested

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

c:\users\nauman\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


45.278752 -66.058044


In [4]:
search_query = 'School'
radius = 500
print(search_query + ' .... OK!')

School .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XMGFAGKBMEINJ4SMM4RTPAPGWBBDBE2S4OOGF1PUFRZJYBU1&client_secret=5PSGNJQPGPQQFQPZB02CJAPVRFHPLRJF52PQTE0Q23FEJANR&ll=45.278752,-66.058044&v=20180604&query=School&radius=500&limit=100'

In [6]:
results = requests.get(url).json() # to 
results

{'meta': {'code': 200, 'requestId': '5c34baca6a60713c2bcabdd1'},
 'response': {'venues': [{'id': '4ec650be6c251306c9fe4309',
    'name': 'Prince Charles School',
    'location': {'lat': 45.275727,
     'lng': -66.054578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.275727,
       'lng': -66.054578}],
     'distance': 432,
     'cc': 'CA',
     'city': 'Saint John',
     'state': 'NB',
     'country': 'Canada',
     'formattedAddress': ['Saint John NB', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d13b941735',
      'name': 'School',
      'pluralName': 'Schools',
      'shortName': 'School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1546959562',
    'hasPerk': False},
   {'id': '4d34713998336dcb21a137f0',
    'name': 'Prince Charles King Edward School',
    'location': {'address': '307-335 Union St',
     'lat': 45.27628986,
     'lng': -66.05270264

In [7]:
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",False,4ec650be6c251306c9fe4309,NaN,CA,Saint John,Canada,432,"[Saint John NB, Canada]","[{'label': 'display', 'lat': 45.275727, 'lng':...",45.275727,-66.054578,NaN,NB,Prince Charles School,v-1546959562
1,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",False,4d34713998336dcb21a137f0,307-335 Union St,CA,Saint John,Canada,500,"[307-335 Union St, Saint John NB E2L 1B3, Canada]","[{'label': 'display', 'lat': 45.27628986, 'lng...",45.276290,-66.052703,E2L 1B3,NB,Prince Charles King Edward School,v-1546959562


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Prince Charles School,School,NaN,CA,Saint John,Canada,432,"[Saint John NB, Canada]","[{'label': 'display', 'lat': 45.275727, 'lng':...",45.275727,-66.054578,NaN,NB,4ec650be6c251306c9fe4309
1,Prince Charles King Edward School,School,307-335 Union St,CA,Saint John,Canada,500,"[307-335 Union St, Saint John NB E2L 1B3, Canada]","[{'label': 'display', 'lat': 45.27628986, 'lng...",45.276290,-66.052703,E2L 1B3,NB,4d34713998336dcb21a137f0


In [9]:
dataframe_filtered.name

0                Prince Charles School
1    Prince Charles King Edward School
Name: name, dtype: object

In [11]:
venue_id = '4ec650be6c251306c9fe4309' # ID of Prince Charles School
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4ec650be6c251306c9fe4309?client_id=XMGFAGKBMEINJ4SMM4RTPAPGWBBDBE2S4OOGF1PUFRZJYBU1&client_secret=5PSGNJQPGPQQFQPZB02CJAPVRFHPLRJF52PQTE0Q23FEJANR&v=20180604'

In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'venueRatingBlacklisted', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes'])


{'id': '4ec650be6c251306c9fe4309',
 'name': 'Prince Charles School',
 'contact': {},
 'location': {'lat': 45.275727,
  'lng': -66.054578,
  'labeledLatLngs': [{'label': 'display',
    'lat': 45.275727,
    'lng': -66.054578}],
  'cc': 'CA',
  'city': 'Saint John',
  'state': 'NB',
  'country': 'Canada',
  'formattedAddress': ['Saint John NB', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/prince-charles-school/4ec650be6c251306c9fe4309',
 'categories': [{'id': '4bf58dd8d48988d13b941735',
   'name': 'School',
   'pluralName': 'Schools',
   'shortName': 'School',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'likes': {'count': 0, 'groups': []},
 'dislike': False,
 'ok': False,
 'venueRatingBlacklisted': True,
 'beenHere': {'count': 0,
  'unconfirmedCount': 0,
  'marked': False,
  'lastCheckinExpiredAt': 0},
 'specials': {'count': 0, 'items': []},
 'photos

In [13]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [14]:
result['response']['venue']['tips']['count']

0